In [3]:
import pandas as pd

In [4]:
train_data = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test_data = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [5]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
# Tokenize the text

max_words = 10000  # Maximum number of words to keep
max_len = 100  # Maximum length of each tweet

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_data['text'])

X_train = tokenizer.texts_to_sequences(train_data['text'])
X_test = tokenizer.texts_to_sequences(test_data['text'])

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [10]:
y_train = train_data['target']

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense

embedding_dim = 100

model = Sequential([
    Embedding(max_words, embedding_dim, input_length=max_len),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [15]:
# Train the model
from tensorflow.keras.callbacks import EarlyStopping 

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs = 10,
    batch_size = 32,
    validation_data = (X_val, y_val),
    callbacks=[early_stopping]
)

Epoch 1/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 22s 89ms/step - accuracy: 0.6605 - loss: 0.6087 - val_accuracy: 0.8135 - val_loss: 0.4290
Epoch 2/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 17s 90ms/step - accuracy: 0.8741 - loss: 0.3220 - val_accuracy: 0.7984 - val_loss: 0.4503
Epoch 3/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 20s 87ms/step - accuracy: 0.9237 - loss: 0.2142 - val_accuracy: 0.7958 - val_loss: 0.5207
Epoch 4/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 21s 90ms/step - accuracy: 0.9416 - loss: 0.1629 - val_accuracy: 0.7669 - val_loss: 0.5838


Model is overfitting!

In [17]:
# Make predictions on test set
preds = model.predict(X_test)
y_pred_binary = (preds>0.5).astype(int)

# Create submission file and submit
submission = pd.DataFrame({
    'id': test_data['id'],
    'target': y_pred_binary.flatten()
})

submission.to_csv('submission.csv', index=False)

102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step
